In [1]:
import requests
import numpy as np
import pandas as pd
import psycopg2
from bs4 import BeautifulSoup
from src.databases import Postgresql

# Criando lista de skus e urls

In [2]:
#Sku
lst_sku=[]
for i in range(1,84):
    PAGE = f'https://www.wine.com.br/vinhos/cVINHOS-p{i}.html'
    if i % 10 == 0:
        print(f'Raspando pagina {i}')

    resultado = requests.get(PAGE)
    bs = BeautifulSoup(resultado.content, 'html.parser')
    html_data = bs.find_all('div', {'class': 'ProductDisplay-name'})
    sku = np.array([elem.a['data-product-sku'] for elem in html_data])
    
    for elem in html_data:
        lst_sku.append(elem.a['data-product-sku'])

Raspando pagina 10
Raspando pagina 20
Raspando pagina 30
Raspando pagina 40
Raspando pagina 50
Raspando pagina 60
Raspando pagina 70
Raspando pagina 80


In [3]:
lista = []
lista_vinhos = []

for i in range(1,84):

    r = requests.get(f'https://www.wine.com.br/browse.ep?cID=100851&exibirEsgotados=false&pn={i}&listagem=horizontal&sorter=featuredProducts-desc&filters=cVINHOS')
    soup = BeautifulSoup(r.content,'html.parser')
    container = soup.find_all('div',{'class':'ProductDisplay-name'})
    for col in container: # contém todos os vinhos
        for art in col.find_all('a',href=True): #vinhos em cada coluna
            lista = 'https://www.wine.com.br{}'.format(art['href'])
            lista_vinhos.append(lista)

In [4]:
lista_vinhos[0:3]

['https://www.wine.com.br/vinhos/maison-le-star-rose-on-ice/prod18479.html',
 'https://www.wine.com.br/vinhos/champagne-montaudon-brut/prod3170.html',
 'https://www.wine.com.br/vinhos/ropiteau-pinot-noir-2017/prod22807.html']

# Extraindo Variavel

In [5]:
harmonizacao = []
for i, j in enumerate(lista_vinhos):
    prods_url = j
    resultado = requests.get(prods_url)
    bs = BeautifulSoup(resultado.content, 'html.parser')
    html_data_harm = bs.find_all('article',{'class': 'TechnicalDetails-matching'})
    
    for elem in html_data_harm:
        harmonizacao.append({'sku':lst_sku[i],'Harmonizacao':elem.dd.string})

dataset_harmonizacao = pd.DataFrame(harmonizacao)

In [6]:
dataset_harmonizacao

,sku,Harmonizacao
0,18479,"Salada de frutos do mar, bruschettas e tapas, ..."
1,3170,"Ostras, salmão defumado, robalo grelhado com p..."
2,22807,"Polenta mole com ragu de linguiça e cogumelos,..."
3,22698,"Camarões grelhados, lagosta, carpaccio, tartar..."
4,21768,"Boeuf Bourguignon, lombo de cordeiro ao molho ..."
...,...,...
860,22659,"Arroz de pato, lombo assado, batata, galeto as..."
861,22659,"Carne vermelha, costela ao molho barbecue, esc..."
862,22659,"Costela bovina assada com especiarias, stinco ..."
863,19479,"Costela de porco ao forno, salada com queijo d..."


In [8]:
query = """ CREATE TABLE harmonizacaov1 ( sku TEXT,
                                            harmonizacao TEXT)  """



bd = Postgresql()
bd.create_table(query=query,user='postgres' , password='bruno22#' , host= 'localhost', port= '5432', database = 'brunods')
bd.insert_data(df = dataset_harmonizacao, tabela = ' harmonizacaov1',user='postgres' , password='bruno22#' , host= 'localhost', port= '5432', database = 'brunods')

Tabela criada no postgresql com sucesso 
Conexão com Postgresql fechada
